In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time 
from pandas import DataFrame

from selenium.webdriver.support.select import Select

In [13]:
def Zhejiang_HuiGouGongGao():
    driver = webdriver.Chrome('../chromedriver')
    driver.get("https://www.zjpwq.net/cms/xxgg/index.htm")
    time.sleep(1) 
    select = Select(driver.find_element_by_id("noticeType"))
    select.select_by_visible_text("回购公告")
    driver.find_element_by_id("search").click()

    time.sleep(1) 
    print("Getting url list ...")
    url_list = []
    while True:
        try:
            driver.find_element_by_css_selector(".layui-laypage-next.layui-disabled")
        except Exception:
            print("---Next page---")
            data_list= driver.find_element_by_class_name("datalist").find_elements_by_class_name("clearfix")
            for item in data_list:
                url_list.append(item.find_element_by_tag_name("a").get_attribute("href"))
            driver.find_element_by_class_name("layui-laypage-next").click()  
            time.sleep(1)
        else:
            print("---No next page---")
            data_list= driver.find_element_by_class_name("datalist").find_elements_by_class_name("clearfix")
            for item in data_list:
                url_list.append(item.find_element_by_tag_name("a").get_attribute("href"))
            print(len(url_list))
            break
    time.sleep(1)        

    result = []
    header = ["发布时间","单位名称","回购总价（元）","最终审批意见","申请回购指标","回购数量","排污权来源","回购单价(元/吨*年)","剩余期限(年)","回购金额(元)","回购数量","排污权来源","回购单价(元/吨*年)","剩余期限(年)","回购金额(元)"]

    for url in url_list:
        driver.get(url)
        time.sleep(1)
        release_date = driver.find_element_by_class_name("Main-HeadContent").find_element_by_tag_name("em").text.split("：")[1]
        table_item = driver.find_element_by_class_name("TableList").find_elements_by_class_name("TableData")
        table_data = []
        for item in table_item:
            table_data.append(item.text)
        for x in header:
            try:
                table_data.remove(x)
            except Exception:
                print('Element not in list')
        row = []
        row.append(release_date)
        row.extend(table_data)
        print(len(row))
        print(url)
        result.append(row)
    file_name = "浙江省排污交易网政府储备量回购信息公告_" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
    DataFrame(result,columns=header).to_csv(file_name)
    print(file_name + ' SAVED')       

In [19]:
driver = webdriver.Chrome('../chromedriver')
driver.get("https://www.zjpwq.net/cms/zrjggs/index.htm")
time.sleep(1)

url_list = []
while True:
    try:
        driver.find_element_by_css_selector(".layui-laypage-next.layui-disabled")
    except Exception:
        print("---Next page---")
        data_list= driver.find_element_by_class_name("datalist").find_elements_by_class_name("clearfix")
        for item in data_list:
            url_list.append(item.find_element_by_tag_name("a").get_attribute("href"))
        driver.find_element_by_class_name("layui-laypage-next").click()  
        time.sleep(1)
    else:
        print("---No next page---")
        data_list= driver.find_element_by_class_name("datalist").find_elements_by_class_name("clearfix")
        for item in data_list:
            url_list.append(item.find_element_by_tag_name("a").get_attribute("href"))
        print(len(url_list))
        break

        
result = []
header = ["发布时间","公示类型","场次名称","指标","出让数量","出让价格","公示时间","企业名称-出让方交易信息","出让数量(吨)-出让方交易信息","成交数量(吨)-出让方交易信息","替代比例-出让方交易信息","项目新增量(吨)-出让方交易信息","出让年限-出让方交易信息","成交均价(元/吨.年)-出让方交易信息","成交总额-出让方交易信息","是否成交-出让方交易信息","企业名称-受让方交易信息","申请数量(吨)-受让方交易信息","成交数量(吨)-受让方交易信息","替代比例-受让方交易信息","项目新增量(吨)-受让方交易信息","成交价格(元/吨.年)-受让方交易信息","出价(摘牌)时间-受让方交易信息","是否竞得-受让方交易信息"]

bad_urls = []
for url in url_list:
    driver.get(url)
    time.sleep(1)

    release_date = driver.find_element_by_class_name("Main-HeadContent").find_element_by_tag_name("em").text.split("：")[1]

    table_item = driver.find_element_by_class_name("TableList").find_elements_by_class_name("TableData")

    table_data = []
    for item in table_item:
        table_data.append(item.text)

    exclude = ["公示类型","场次名称","指标","出让数量","出让价格","公示时间","企业名称","出让数量(吨)","成交数量(吨)","替代比例","项目新增量(吨)","出让年限","成交均价(元/吨.年)","成交总额","是否成交","企业名称","申请数量(吨)","成交数量(吨)","替代比例","项目新增量(吨)","成交价格(元/吨.年)","出价(摘牌)时间","是否竞得","受让方交易信息","出让方交易信息"]
    exclude_count = 0
    for x in exclude:
        try:
            table_data.remove(x)
        except Exception:
            print('Element not in list')
            exclude_count += 1

    row = []
    row.append(release_date)
    row.extend(table_data)
    print(len(row))
    print(exclude_count)
    if exclude_count == 0: # no missing header
        result.append(row)
    else:
        print(url)
        bad_urls.append([url, exclude_count])

file_name = "浙江省排污交易网企业排污权转让_结果公示_part1" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
DataFrame(result,columns=header).to_csv(file_name)
print(file_name + ' SAVED')  



---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next page---
---Next 

Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
27
5
https://www.zjpwq.net/cms/zrjggs/24965.htm
Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
15
5
https://www.zjpwq.net/cms/zrjggs/24745.htm
Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
15
5
https://www.zjpwq.net/cms/zrjggs/24744.htm
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
24
0
Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
21
5
https://www.zjpwq.net/cms/zrjggs/239

Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21530.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21519.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21518.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21517.htm
24
0
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21607.htm
Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
15
5
https://www.zjpwq.net/cms/zrjggs/21574.htm
Element not in list
Element not in list
Element not in list
Element not in list
Element not in list
15
5
https://www.zjpwq.net/cms/zrjggs/21573.htm
Element not in list
Element not in list
Element not in list

Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21327.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21315.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21291.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21290.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21289.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21288.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21287.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.n

Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21064.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21063.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21062.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21061.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21060.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21056.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.net/cms/zrjggs/21055.htm
Element not in list
Element not in list
Element not in list
Element not in list
20
4
https://www.zjpwq.n